In [1]:
!pip install texthero

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 6.7 MB/s 


In [2]:
import nltk
from nltk.collocations import *
import texthero as hero
import pandas as pd
import itertools
import numpy as np 
from nltk import bigrams 
from nltk.corpus import inaugural
nltk.download('genesis')
nltk.download('inaugural')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data]   Unzipping corpora/inaugural.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
print("Number of words",len(inaugural.words("/content/drive/MyDrive/Colab Notebooks/NLP/mughal.txt")))
corpus=(inaugural.words("/content/drive/MyDrive/Colab Notebooks/NLP/mughal.txt"))
print(corpus)

Number of words 24104
['Mughal', 'Empire', 'From', 'Wikipedia', ',', 'the', ...]


Text Processing

In [7]:
corpus=pd.Series(corpus)
corpus=hero.remove_punctuation(corpus)   #Remove all string.punctuation (!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~).
corpus=hero.remove_diacritics(corpus)    #Remove all accentuated characters from strings.
corpus=hero.remove_stopwords(corpus)      #Remove all stop words.
corpus=hero.remove_whitespace(corpus)    #Remove whitespace
corpus=hero.lowercase(corpus)              #Lowercase all text.
corpus=hero.remove_digits(corpus)         #Remove all blocks of digits.


Top 50 bigrams

In [8]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(corpus)
finder.nbest(bigram_measures.pmi, 50)

[('abhishek', 'kaicker'),
 ('abjuring', 'liquor'),
 ('adroitly', 'gave'),
 ('aerial', 'view'),
 ('aerospace', 'laboratories'),
 ('afghanistanmedieval', 'indiahistorical'),
 ('aggregated', 'millet'),
 ('aggrieved', 'subject'),
 ('allowed', 'freedom'),
 ('american', 'ethnologist'),
 ('amir', 'khusrau'),
 ('ancient', 'greece'),
 ('and', 'designing'),
 ('articletalk', 'readview'),
 ('ashok', 'desai'),
 ('assamese', 'highlands'),
 ('assesses', 'ship'),
 ('attracting', '7a'),
 ('autumn', 'festivals'),
 ('banarsi', 'prasad'),
 ('best', 'guidance'),
 ('beyond', 'repair'),
 ('bibi', 'ka'),
 ('boo', 'rods'),
 ('brass', 'utensils'),
 ('brother', 'gommans'),
 ('cadastral', 'surveying'),
 ('chandrika', 'kaul'),
 ('chapai', 'nawabganj'),
 ('chroniclers', 'bewailed'),
 ('claimed', 'ultimate'),
 ('colin', 'mcevedy'),
 ('concerning', 'mankind'),
 ('continual', 'pacification'),
 ('contraction', 'patterns'),
 ('contribute', 'help'),
 ('conventionally', 'reckoned'),
 ('corps', 'consisting'),
 ('courtier',

In [30]:
def pmi(df):
    '''
    Calculate the positive pointwise mutal information score for each entry
    https://en.wikipedia.org/wiki/Pointwise_mutual_information
    We use the log( p(y|x)/p(y) ), y being the column, x being the row
    '''
    # Get numpy array from pandas df
    #arr = df.as_matrix()
    arr=np.array(df)
    # p(y|x) probability of each t1 overlap within the row
    row_totals = arr.sum(axis=1).astype(float)
    prob_cols_given_row = (arr.T / row_totals).T

    # p(y) probability of each t1 in the total set
    col_totals = arr.sum(axis=0).astype(float)
    prob_of_cols = col_totals / sum(col_totals)

    # PMI: log( p(y|x) / p(y) )
    # This is the same data, normalized
    ratio = prob_cols_given_row / prob_of_cols
    ratio[ratio==0] = 0.00001
    _pmi = np.log(ratio)
    _pmi[_pmi < 0] = 0

    return _pmi


In [27]:
   

def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)

    # Key:Value = Word:Index
    vocab_to_index = { word:i for i, word in enumerate(vocab) }

    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))

    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    # Initialise co-occurrence matrix
    # co_occurrence_matrix[current][previous]
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

    # Loop through the bigrams in the frequency distribution, noting the 
    # current and previous word, and the number of occurrences of the bigram.
    # Get the vocab index of the current and previous words.
    # Put the number of occurrences into the appropriate element of the array.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_to_index[current]
        pos_previous = vocab_to_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count 

    co_occurrence_matrix = np.matrix(co_occurrence_matrix)

    return co_occurrence_matrix

test_sent =finder.nbest(bigram_measures.pmi,10)
m = co_occurrence_matrix(corpus)

In [28]:
print(m)

[[2.03e+03 1.00e+00 0.00e+00 ... 0.00e+00 1.00e+00 2.00e+00]
 [1.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [1.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 ...
 [0.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [1.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [1.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]]


PPMI

In [31]:
PPMI=pmi(m)
print(PPMI)

[[0.         0.86427165 0.         ... 0.         0.86427165 0.86427165]
 [0.86437012 0.         0.         ... 0.         0.         0.        ]
 [0.17122294 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.86437012 0.         0.         ... 0.         0.         0.        ]
 [0.17122294 0.         0.         ... 0.         0.         0.        ]]
